In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
from sklearn.metrics import roc_auc_score

import fastai
from fastai.vision import *

# Any results you write to the current directory are saved as output.

In [ ]:
df = pd.read_csv('../input/traininglabels.csv')
df_sample = pd.read_csv('../input/SampleSubmission.csv')
df[df['score']<0.75]

In [ ]:
test = Path('../input/leaderboard_test_data/leaderboard_test_data')
holdout = Path('../input/leaderboard_holdout_data/leaderboard_holdout_data')
test_names = [f for f in test.iterdir()]
holdout_names = [f for f in holdout.iterdir()]

In [ ]:
src = (ImageItemList.from_df(df, '../input/', folder='train_images/train_images')
      .random_split_by_pct(0.2, seed=50)
      .label_from_df('has_oilpalm')
      .add_test(test_names+holdout_names))

#split = 0.2, seed = 2019

In [ ]:
data =  (src.transform(get_transforms(), size=128)
         .databunch(bs=64)
         .normalize(imagenet_stats))

In [ ]:
from sklearn.metrics import roc_auc_score
def auc_score(preds,targets):
    return torch.tensor(roc_auc_score(t,p[:,1]))

In [ ]:
learn = create_cnn(data, models.resnet50, 
                   metrics=[accuracy,auc_score],
                   model_dir='/kaggle/working/models')

In [ ]:
learn.lr_find(); learn.recorder.plot()
lr = 1e-2
learn.fit_one_cycle(6, lr)
learn.unfreeze()
learn.fit_one_cycle(3, slice(1e-4, 1e-3))
learn.save('128')
p,t = learn.get_preds()
#auc_score(p,t)


In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
losses,idxs = interp.top_losses()

In [ ]:
#interp.plot_top_losses(9, figsize=(15,11))

In [ ]:
p,t = learn.get_preds(ds_type=DatasetType.Test)
p = to_np(p);

#check with p.shape

ids = np.array([f.name for f in (test_names+holdout_names)]);ids.shape
answerdf = pd.DataFrame(np.stack([ids, p[:,1]], axis=1), columns=df_sample.columns)
answerdf.to_csv('SubmFastAiResnet50_v2.csv', index=False)